# Imports

In [1]:
from ast import literal_eval
import functools
import json
import os
import random
import shutil

# Scienfitic packages
import numpy as np
import pandas as pd
import torch
import datasets
torch.set_grad_enabled(False)

# Visuals
from matplotlib import pyplot as plt
import seaborn as sns
sns.set(context="notebook",
        rc={"font.size":16,
            "axes.titlesize":16,
            "axes.labelsize":16,
            "xtick.labelsize": 16.0,
            "ytick.labelsize": 16.0,
            "legend.fontsize": 16.0})
palette_ = sns.color_palette("Set1")
palette = palette_[2:5] + palette_[7:]
sns.set_theme(style='whitegrid')

# Utilities

from general_utils import (
  ModelAndTokenizer,
  make_inputs,
  decode_tokens,
  find_token_range,
  predict_from_input,
)

from patchscopes_utils import *

from tqdm import tqdm
tqdm.pandas()

In [2]:
model_to_hook = {
    "EleutherAI/pythia-12b": set_hs_patch_hooks_neox,
    "meta-llama/Llama-2-13b-hf": set_hs_patch_hooks_llama,
    "lmsys/vicuna-7b-v1.5": set_hs_patch_hooks_llama,
    "./stable-vicuna-13b": set_hs_patch_hooks_llama,
    "CarperAI/stable-vicuna-13b-delta": set_hs_patch_hooks_llama,
    "EleutherAI/gpt-j-6b": set_hs_patch_hooks_gptj
}

In [ ]:
# Load model

model_name = "EleutherAI/pythia-12b"
sos_tok = False

if "13b" in model_name or "12b" in model_name:
    torch_dtype = torch.float16
else:
    torch_dtype = None

mt = ModelAndTokenizer(
    model_name,
    low_cpu_mem_usage=False,
    torch_dtype=torch_dtype,
)
mt.set_hs_patch_hooks = model_to_hook[model_name]
mt.model.eval()

C:\Users\NickName\miniconda3\envs\patchscopes\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NickName\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [ ]:
df = pd.read_csv("popQA.tsv", sep="\t")
df.columns

In [ ]:
tmp = pd.concat([
    df[["subj", "s_wiki_title", "s_pop"]].drop_duplicates().sort_values(by="s_pop").head(200),
    df[["subj", "s_wiki_title", "s_pop"]].drop_duplicates().sort_values(by="s_pop").tail(200)
])
tmp.shape

In [ ]:
generation_mode = True
max_gen_len = 50

# prompt_source = "Devalan" # "Jurassic Park" # "Back to the Future" # "New York City" #"Muhammad Ali" # "Alexander the Great" #"Diana, Princess of Wales"
prompt_target = f"Syria: Country in the Middle East, Leonardo DiCaprio: American actor, Samsung: South Korean multinational major appliance and consumer electronics corporation, x"
position_source = -1
position_target = -1


for layer_source in range(10):
    layer_target = layer_source

    tmp[f"inspect_layer{layer_source}"] = tmp.subj.progress_apply(
        lambda prompt_source: inspect(mt, prompt_source, prompt_target, layer_source, layer_target,
                                      position_source, position_target,
                                      generation_mode=generation_mode, max_gen_len=max_gen_len, verbose=False))

tmp.to_csv("popqa_200-200_input_processing_vicuna-13b-v1.1.csv", index=False)